In [1]:
import pandas as pd
import numpy as np

In [2]:
# These functions prune our frontier (of players p,v combos) by checking to see if they are strictly dominated.
# This means it checks to see if we can get the same value at a lower price, or higher value at the same price, we remove the node.
# This results in a strictly increasing relationship between price and value and could be graphed

def helper(val, d, up):
    if up:
        return max([v for k, v in d.iteritems() if k <= val])
    else:
        return min([v for k, v in d.iteritems() if k >= val])        

def dominated(price, value):
    prices = np.unique(price)
    values = np.unique(value)
    best_vals = {p : max([value[i] for i, v in enumerate(price) if v == p]) for p in prices}
    best_pris = {v : min([price[i] for i, p in enumerate(value) if p == v]) for v in values}
    bests = [(k,v) for k, v in best_vals.iteritems()]+[(v,k) for k, v in best_pris.iteritems()]
    bests_ = [(p,v) for p, v in bests if (v >= helper(p, best_vals, True)) and (p <= helper(v, best_pris, False))]
    return set(bests_)

In [3]:
# This function checks for each value if it has the proper number of players, and replaces it with 0 if it is impossible (meaning 
# that there are no combos to be at or below that price), or replaces it with the highest achievable value at some lower price
# with the correct number of players.

def clean(d, val):
    lst = []
    for k, v in d.iteritems():
        if len(v[1]) == val+1:
            lst.append(v)
        else:
            try:
                d[k] = lst[-1]
            except IndexError:
                d[k] = (0,[])
    return d

In [22]:
#  This goes by position and creates new price, value dict.  Has the option to require a player is added from each position. 
def add_pos(in_d, vals, require=False):
    out_d = in_d.copy()
    val = len(in_d[max(in_d.keys())][1])
    for p, s in in_d.iteritems():
        for p_, v in vals:
            if p+p_ <= max(in_d.keys()):
                if out_d[p+p_][0] < in_d[p][0]+v or (require and val >= len(out_d[p+p_][1])):
                    out_d[p+p_] = (in_d[p][0]+v, in_d[p][1]+[(p_, v)])
    if require:
        out_d = clean(out_d, val)
    return out_d

In [23]:
# Set our Price Ceiling and subdivisions of a dollar, here we go by .1 or a tenth and create a price dictionary
price_ceiling = 50
density = 10
prices = {v : (0,[]) for v in range(price_ceiling*density+1)} 

In [24]:
# Number of Random Players to Generate of form (Position, Value, Price) where Position is 1-5, Value is 1-51,
# and Price is 1-50 (but is 2/3rds based on Value)
n = 10000
pos = np.random.choice(range(1, 6), n)
value = np.random.choice(range(1, 511), n)
price = [2*x/3 + y for x, y in zip(value, np.random.choice(range(1, 161), n))]

players = zip(pos, price, value)
#players = [(1,100,49.9), (1,1,9.1), (2,2,9.2), (3,3,9.3), (4,4,9.4), (5,91,9.5)]

In [25]:
# Create a player dictionary by position with the form {Pos : [(Val, Price), (V2,P2), ...]}
pos_d = {}
for p in players:
    try:
        pos_d[p[0]].append((p[1], p[2]))
    except:
        pos_d[p[0]] = [(p[1], p[2])]

In [26]:
# Print out number of players at each position
for k, v in pos_d.iteritems():
    print k, len(v)

1 1998
2 1992
3 2008
4 1981
5 2021


In [34]:
%%time
in_d = prices.copy()
for p in pos_d.keys():
    in_d = add_pos(in_d, list(dominated(*zip(*pos_d[p]))), require=False)

Wall time: 4.25 s


In [35]:
in_d

{0: (0, []),
 1: (0, []),
 2: (2, [(2, 2)]),
 3: (2, [(2, 2)]),
 4: (2, [(2, 2)]),
 5: (2, [(2, 2)]),
 6: (2, [(2, 2)]),
 7: (9, [(7, 9)]),
 8: (10, [(8, 10)]),
 9: (13, [(9, 13)]),
 10: (13, [(9, 13)]),
 11: (15, [(2, 2), (9, 13)]),
 12: (15, [(12, 15)]),
 13: (17, [(2, 2), (11, 15)]),
 14: (17, [(2, 2), (12, 15)]),
 15: (21, [(15, 21)]),
 16: (22, [(9, 13), (7, 9)]),
 17: (23, [(2, 2), (15, 21)]),
 18: (24, [(2, 2), (9, 13), (7, 9)]),
 19: (25, [(2, 2), (8, 10), (9, 13)]),
 20: (28, [(20, 28)]),
 21: (28, [(20, 28)]),
 22: (30, [(2, 2), (20, 28)]),
 23: (31, [(8, 10), (15, 21)]),
 24: (35, [(24, 35)]),
 25: (35, [(24, 35)]),
 26: (37, [(2, 2), (24, 35)]),
 27: (37, [(2, 2), (24, 35)]),
 28: (38, [(8, 10), (20, 28)]),
 29: (41, [(20, 28), (9, 13)]),
 30: (41, [(20, 28), (9, 13)]),
 31: (43, [(2, 2), (20, 28), (9, 13)]),
 32: (46, [(32, 46)]),
 33: (48, [(9, 13), (24, 35)]),
 34: (48, [(2, 2), (32, 46)]),
 35: (50, [(35, 50)]),
 36: (51, [(36, 51)]),
 37: (55, [(37, 55)]),
 38: (55, [(

In [33]:
in_d

{0: (0, []),
 1: (0, []),
 2: (0, []),
 3: (0, []),
 4: (0, []),
 5: (0, []),
 6: (0, []),
 7: (0, []),
 8: (0, []),
 9: (0, []),
 10: (0, []),
 11: (0, []),
 12: (0, []),
 13: (0, []),
 14: (0, []),
 15: (0, []),
 16: (0, []),
 17: (0, []),
 18: (0, []),
 19: (0, []),
 20: (0, []),
 21: (0, []),
 22: (0, []),
 23: (0, []),
 24: (0, []),
 25: (0, []),
 26: (0, []),
 27: (0, []),
 28: (0, []),
 29: (0, []),
 30: (0, []),
 31: (0, []),
 32: (0, []),
 33: (0, []),
 34: (0, []),
 35: (35, [(2, 2), (8, 10), (9, 1), (9, 13), (7, 9)]),
 36: (35, [(2, 2), (8, 10), (9, 1), (9, 13), (7, 9)]),
 37: (35, [(2, 2), (8, 10), (9, 1), (9, 13), (7, 9)]),
 38: (37, [(2, 2), (8, 10), (9, 1), (12, 15), (7, 9)]),
 39: (47, [(2, 2), (8, 10), (13, 13), (9, 13), (7, 9)]),
 40: (47, [(2, 2), (8, 10), (13, 13), (9, 13), (7, 9)]),
 41: (47, [(2, 2), (8, 10), (13, 13), (9, 13), (7, 9)]),
 42: (49, [(2, 2), (8, 10), (13, 13), (12, 15), (7, 9)]),
 43: (53, [(2, 2), (8, 10), (13, 13), (9, 13), (11, 15)]),
 44: (53, [